In [2]:
!pip install kfp minio pandas boto3

  Using cached pandas-2.2.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached s3transfer-0.11.4-py3-none-any.whl.metadata (1.7 kB)
Using cached pandas-2.2.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.1 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.6/139.6 kB 1.2 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 18.4 MB/s eta 0:00:0000:0100:01
Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.4 MB)
Using cached s3transfer-0.11.4-py3-none-any.whl (84 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)


In [24]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
import boto3
import os
import shutil
from datetime import datetime
from pyspark.sql.functions import lit, col, when

# variables
# variables
# access_key_id = 'admin'
# secret_access_key = 'dlyticaD123'
# minio_endpoint = 'dn-minio-tenant-hl.dn-minio-tenant.svc.cluster.local:9000'  # Replace with your MinIO server endpoint
# data_bucket = 'ai360fd-recommendation'
# apps_bucket = ''
# hive_metastore_uri = "thrift://dn-hive-metastore.dn-hive-metastore.svc.cluster.local:9083"
# iceberg_warehouse_location = f"s3a://{data_bucket}/data/"
# custom_catalog = "iceberg_catalog"
# spark_eventlog_dir = f"s3a://{apps_bucket}/logs/spark/"
# master_url="spark://dn-spark-master-svc.dn-spark.svc.cluster.local:7077"
# spark_driver_host="notebook-five-spark-driver-headless.kubeflow-user-example-com.svc.cluster.local"
# app_name="All Data Generation"
# additional_packages="org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.4.2,org.apache.spark:spark-avro_2.12:3.5.0,com.crealytics:spark-excel_2.12:0.13.7,org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.12.262,org.apache.hadoop:hadoop-common:3.3.4"

access_key_id = 'admin'
secret_access_key = 'dlyticaD123'
minio_endpoint = 'dn-minio-tenant-hl.dn-minio-tenant.svc.cluster.local:9000'  # Replace with your MinIO server endpoint
data_bucket = 'ai360fd-recommendation'
# apps_bucket = 'dn-apps'
hive_metastore_uri = "thrift://dn-hive-metastore.dn-hive-metastore.svc.cluster.local:9083"
iceberg_warehouse_location = f"s3a://{data_bucket}/data/"
custom_catalog = "iceberg_catalog"
# spark_eventlog_dir = f"s3a://{apps_bucket}/logs/spark/"
# master_url="spark://dn-spark-master-svc.dn-spark.svc.cluster.local:7077"
# spark_driver_host="notebook-five-spark-driver-headless.kubeflow-user-example-com.svc.cluster.local"
app_name="All Data Generation"
additional_packages="org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.4.2,org.apache.spark:spark-avro_2.12:3.5.0,com.crealytics:spark-excel_2.12:0.13.7,org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.12.262,org.apache.hadoop:hadoop-common:3.3.4"

spark = SparkSession.builder \
    .appName(app_name) \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .config("spark.executor.memoryOverhead", "2g") \
    .config("spark.driver.maxResultSize", "2g") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.endpoint", minio_endpoint) \
    .config("spark.hadoop.fs.s3a.access.key", access_key_id) \
    .config("spark.hadoop.fs.s3a.secret.key", secret_access_key) \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .config("spark.hadoop.hive.metastore.uris", hive_metastore_uri) \
    .config("spark.sql.warehouse.dir", iceberg_warehouse_location) \
    .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog") \
    .config(f"spark.sql.catalog.{custom_catalog}", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.jars.packages", additional_packages) \
    .config("spark.sql.repl.eagerEval.enabled", True) \
    .config("spark.sql.debug.maxToStringFields", 1000) \
    .enableHiveSupport() \
    .getOrCreate()

# spark.sql(f"""
# show tables in {custom_catalog}.gold
# """).show(100,False)
# print("train_model is called")
# Initialize Spark session
# spark = SparkSession.builder.appName("Feature Engineering").getOrCreate()


25/04/07 09:33:25 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [25]:
spark.sql(f"""
show tables in {custom_catalog}.gold
""").show(100,False)

+---------+---------------------------------+-----------+
|namespace|tableName                        |isTemporary|
+---------+---------------------------------+-----------+
|gold     |customer                         |false      |
|gold     |dim_mob_profile                  |false      |
|gold     |dim_mob_request_info             |false      |
|gold     |dim_mob_bank_account             |false      |
|gold     |dim_mob_customer_login           |false      |
|gold     |dim_mob_application_user         |false      |
|gold     |dim_mob_customers                |false      |
|gold     |dim_mob_merchant                 |false      |
|gold     |dim_communications               |false      |
|gold     |fact_eod_balance_details         |false      |
|gold     |fact_interest_rate_change_history|false      |
|gold     |dim_mob_logged_in_activity       |false      |
|gold     |dim_lien                         |false      |
|gold     |currency_rates                   |false      |
|gold     |fac

In [28]:
spark.sql(f'SELECT * from  {custom_catalog}.gold.customer_segments')

25/04/07 09:36:06 WARN CheckAllocator: More than one DefaultAllocationManager on classpath. Choosing first found


cif_id,cluster_id,intreapretation
R000372359,6,Moderate Spender;...
R000371858,4,Moderate Spender;...
R000372322,6,Moderate Spender;...
R000371446,6,Moderate Spender;...
R000370704,6,Moderate Spender;...
R000371574,3,Moderate Spender;...
R000372430,6,Moderate Spender;...
R000370804,9,Moderate Spender;...
R000371010,1,Moderate Spender;...
R000371018,6,Moderate Spender;...


In [14]:
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS {custom_catalog}.gold.customer_segments (
            cif_id VARCHAR(50),
            cluster_id INTEGER,
            intrepretation VARCHAR(300)
        )
""")

""


In [ ]:
results -> csv -> 04_03.csv
csv -> read 04_03.csv -> iceberg write (overwrite)
trino table
move csv -> 04_03.csv -> archive folder


first write -> 04_02 -> csv
csv spark read -> iceberg
Move csv -> archive
csv -> empty
archive -> 04_02

04_03 -> csv


In [17]:
spark

In [18]:
input_s3_path = f"s3a://ai360ctzn-customer-segmentation/results/final_segmented_customers_2025-04-02.csv"
 
   #read from a folder in s3 bucket
df = spark.read.csv(input_s3_path, header=True,inferSchema=True)
df.count()

1268538

In [19]:
df.show(5,False)

+----------+-------+----------------------------------------------------------------------------------------------------------------------------+
|cif_id    |cluster|interpretations                                                                                                             |
+----------+-------+----------------------------------------------------------------------------------------------------------------------------+
|C000000056|2      |Moderate Spender; Low Credit Utilization; Aggressive Overpayer; Low Financial Risk                                          |
|C000000291|1      |Moderate Spender; Very High Credit Utilization; Aggressive Overpayer; Extremely High Financial Risk                         |
|C000000586|0      |High Spender; Intensive Cash Advance User; Very High Credit Utilization; Aggressive Overpayer; Extremely High Financial Risk|
|C000000660|1      |Moderate Spender; Very High Credit Utilization; Aggressive Overpayer; Extremely High Financial Risk     

In [20]:
df.writeTo(f"{custom_catalog}.gold.customer_segments1").createOrReplace()

In [ ]:
#selecting  cif_id and predictions from the dataframe
df_selected = predictions.select("cif_id", "prediction")
 
# writin into the predictions of gold schema in iceberg catalog format
df_selected.writeTo(f"{custom_catalog}.gold.predictions").createOrReplace()

In [16]:
# # After creating the connection and cursor, add this:
# cursor.execute("SELECT column_name FROM information_schema.columns WHERE table_schema = 'gold' AND table_name = 'customer_segments'")
# columns = cursor.fetchall()
# print("Available columns:", columns)

NameError: name 'cursor' is not defined

In [ ]:
@component(
    base_image='quay.io/datanature_dev/jupyternotebook:java_home14',
    packages_to_install=[
        'trino',
        'pandas<2.0.0',
        'boto3',
        'urllib3',
        'sqlalchemy',
        'sqlalchemy-trino',
        'pyarrow==12.0.1',
        'numpy<2.0.0'
    ]
)

def save_predictions_to_trino(
    segmentation_path: str,
    minio_bucket: str
) -> str:
    import trino
    from trino.auth import BasicAuthentication
    import pandas as pd
    import boto3
    import urllib3
    import os
    import time
    from io import StringIO
    from concurrent.futures import ThreadPoolExecutor, as_completed

    # Disable SSL warnings
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

    try:
        # Handle input path
        object_key = segmentation_path.split(minio_bucket + '/')[-1] if minio_bucket in segmentation_path else segmentation_path
        print(f"MinIO bucket: {minio_bucket}")
        print(f"Object key: {object_key}")
        
        # 1. Download CSV from MinIO
        minio_client = boto3.client(
            's3',
            endpoint_url="http://192.168.80.155:32000",
            aws_access_key_id="admin",
            aws_secret_access_key="dlyticaD123",
            verify=False,
            config=boto3.session.Config(
                connect_timeout=30,
                read_timeout=60,
                retries={'max_attempts': 3}
            )
        )
        
        local_path = "/tmp/predictions.csv"
        print("Downloading file from MinIO...")
        minio_client.download_file(minio_bucket, object_key, local_path)
        
        # 2. Read CSV in chunks to handle large files
        print("Reading CSV file...")
        chunksize = 100000
        dfs = []
        for chunk in pd.read_csv(local_path, chunksize=chunksize):
            # Rename columns to match Trino table
            chunk = chunk.rename(columns={
                "cluster": "cluster_id",
                "interpretations": "intreapretation"  # Matching the table column
            })
            dfs.append(chunk)
        
        df = pd.concat(dfs)
        print(f"Read {len(df)} records from CSV")
        
        # 3. Connect to Trino
        trino_host = "192.168.80.155"
        trino_port = "30071"
        trino_user = "ctzn.bank"
        trino_password = "ctzn.bank_123"
        trino_catalog = "iceberg"
        trino_schema = "gold"
        
        # Create connection with timeout
        conn = trino.dbapi.connect(
            host=trino_host,
            port=trino_port,
            user=trino_user,
            auth=BasicAuthentication(trino_user, trino_password),
            catalog=trino_catalog,
            schema=trino_schema,
            http_scheme="https",
            verify=False,
            request_timeout=600  # 10 minute timeout
        )
        
        cursor = conn.cursor()
        
        # 4. Prepare table (create if not exists)
        print("Preparing table...")
        cursor.execute(f"""
        CREATE TABLE IF NOT EXISTS iceberg.gold.customer_segments (
            cif_id VARCHAR,
            cluster_id INTEGER,
            intreapretation VARCHAR
        )
        """)
        
        # 5. Clear existing data
        print("Clearing existing data...")
        cursor.execute("DELETE FROM iceberg.gold.customer_segments")
        
        # 6. Insert data in parallel batches
        print("Starting data insertion...")
        batch_size = 5000
        total_rows = len(df)
        batches = [(i, min(i + batch_size, total_rows)) 
                  for i in range(0, total_rows, batch_size)]
        
        def insert_batch(start, end):
            batch_conn = trino.dbapi.connect(
                host=trino_host,
                port=trino_port,
                user=trino_user,
                auth=BasicAuthentication(trino_user, trino_password),
                catalog=trino_catalog,
                schema=trino_schema,
                http_scheme="https",
                verify=False
            )
            batch_cursor = batch_conn.cursor()
            
            try:
                # Prepare batch data
                batch_df = df.iloc[start:end]
                values = []
                for _, row in batch_df.iterrows():
                    values.append((
                        str(row['cif_id']) if pd.notna(row['cif_id']) else None,
                        int(row['cluster_id']) if pd.notna(row['cluster_id']) else None,
                        str(row['intreapretation']) if pd.notna(row['intreapretation']) else None
                    ))
                
                # Build query
                query = "INSERT INTO iceberg.gold.customer_segments (cif_id, cluster_id, intreapretation) VALUES "
                query += ", ".join(["(%s, %s, %s)"] * len(values))
                query = query % tuple([item for sublist in values for item in sublist])
                
                # Execute
                batch_cursor.execute(query)
                return (start, end, True)
            except Exception as e:
                print(f"Batch {start}-{end} failed: {str(e)}")
                return (start, end, False)
            finally:
                batch_cursor.close()
                batch_conn.close()
        
        # Use ThreadPoolExecutor for parallel inserts
        success_count = 0
        with ThreadPoolExecutor(max_workers=4) as executor:
            futures = [executor.submit(insert_batch, start, end) for start, end in batches]
            
            for future in as_completed(futures):
                start, end, success = future.result()
                if success:
                    success_count += 1
                    print(f"Successfully inserted batch {start}-{end}")
                else:
                    print(f"Failed to insert batch {start}-{end}")
        
        # Verify insertion
        print("Verifying data...")
        cursor.execute("SELECT COUNT(*) FROM iceberg.gold.customer_segments")
        count = cursor.fetchone()[0]
        print(f"Verified {count} records in Trino table")
        
        # Close connections
        cursor.close()
        conn.close()
        
        if success_count == len(batches):
            return f"Successfully loaded {count} records to Trino table iceberg.gold.customer_segments"
        else:
            raise Exception(f"Partial success: {success_count}/{len(batches)} batches inserted")
            
    except Exception as e:
        print(f"Error in save_predictions_to_trino: {str(e)}")
        raise
    finally:
        # Clean up
        if 'local_path' in locals() and os.path.exists(local_path):
            os.remove(local_path)